In [ ]:
import torch 
import torch.nn as nn


d:\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class View(nn.Module):
    def __init__(self,shape):
        super().__init__()
        self.shape = (shape,)
    def forward(self,x):
        return x.view(*self.shape)
    

In [3]:
class Name(nn.Module):
    def __init__(self,resl):
        super().__init__()
        
    def forward(self,x): 
        pass
        

In [4]:
class mySequential(nn.Sequential):
    def __init__(self,):
        super().__init__()

    def forward(self,*x): 
        for layer in self.children():
            if type(x) is tuple:
                x = layer(*x)
            else:
                x= layer(x)
        return x
        

## equal LR 분석

In [6]:
import math

class EqualLR:
    def __init__(self):
        pass

    def compute_weight(self,module):
        # num_fanin = 
        weight = getattr(module,'weight_orig1')
        fan_in = weight.data.size(1) * weight.data[0][0].numel()

        return weight**16 * math.sqrt(2 / fan_in)

    def __call__(self, module,input):
        computed_weight = self.compute_weight(module)
        setattr(module,'weight',computed_weight)
        

fn = EqualLR()
module = nn.Linear(3,5)
module.bias = torch.nn.Parameter(torch.FloatTensor(5).fill_(0))
module.register_forward_pre_hook(fn)
weight = getattr(module,'weight')
del module._parameters['weight']
module.register_parameter('weight_orig1', nn.Parameter(weight.data))
#setattr(module,'weight_orig2',weight)


In [7]:
module.__dict__

{'training': True,
 '_parameters': OrderedDict([('bias', Parameter containing:
               tensor([0., 0., 0., 0., 0.], requires_grad=True)),
              ('weight_orig1',
               Parameter containing:
               tensor([[ 0.3844,  0.2602, -0.5109],
                       [-0.0124, -0.3961,  0.1186],
                       [-0.2179, -0.3348, -0.2991],
                       [-0.4081, -0.3056,  0.2651],
                       [-0.2233, -0.0914, -0.5209]], requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict([(0, <__main__.EqualLR at 0x1a96318a3d0>)]),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 'in_features': 3,
 'out_features': 5}

In [8]:
dummy = torch.ones((2,3))
module(dummy)
module.__dict__


{'training': True,
 '_parameters': OrderedDict([('bias', Parameter containing:
               tensor([0., 0., 0., 0., 0.], requires_grad=True)),
              ('weight_orig1',
               Parameter containing:
               tensor([[ 0.3844,  0.2602, -0.5109],
                       [-0.0124, -0.3961,  0.1186],
                       [-0.2179, -0.3348, -0.2991],
                       [-0.4081, -0.3056,  0.2651],
                       [-0.2233, -0.0914, -0.5209]], requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict([(0, <__main__.EqualLR at 0x1a96318a3d0>)]),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 'in_features': 3,
 'out_features': 5,
 'weight': tensor([[1.8537e-07, 3.5964e-10, 1.7566e-05],

styleGAN architecture

In [76]:
def conv1(resl=3):
    layer =[]
    up = nn.Upsample(scale_factor=2)
    if resl>=6:
        conv1 = nn.Conv2d(2**(15-resl),2**(14-resl),3,1,1)
    else:
        conv1 = nn.Conv2d(512,512,3,1,1)
    layer.append(up)
    layer.append(conv1)

    return nn.Sequential(*layer)

def conv2(resl=3):
    layer =[]
    if resl>=6:
        conv1 = nn.Conv2d(2**(14-resl),2**(14-resl),3,1,1)
    else:
        conv1 = nn.Conv2d(512,512,3,1,1)
    layer.append(conv1)

    return nn.Sequential(*layer)

class Noise(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        batch = x.shape[0]
        size = x.size(3)
        noise = torch.randn(batch, 1, size, size)
        print('noise',noise.shape)
        print('noise success')
        return x+noise
class AdaptiveInstanceNorm(nn.Module):
    def __init__(self,ch):
        super().__init__()
        self.ch = ch
        self.instance_norm = nn.InstanceNorm2d(ch)
        self.linear = nn.Linear(512,ch*2)
    def forward(self,x,style_latent):
        batch = x.shape[0]
        size = x.size(3)
        x_norm = self.instance_norm(x)
        ## batch 전부 같은 스타일 넣는 게 맞나??
        style_out = self.linear(style_latent) ## style_latent도 batch size 만큼
        gamma = style_out[:,:self.ch].unsqueeze(2).unsqueeze(3)
        beta = style_out[:,self.ch:].unsqueeze(2).unsqueeze(3)
        print('x_norm',x_norm.shape)
        print('gamma',gamma.shape)
        print('beta',beta.shape)

        print('AdaIN success')
        sample = gamma*x_norm+beta
        print('sample',sample.shape)
        return gamma*x_norm+beta

## StyleConvBlock만들기

In [77]:
#StyleConvBlock만들기
class StyleConvBlock(nn.Module):
    def __init__(self,resl):
        super().__init__()
        if resl >=6:
            ch = 2**(14-resl)
        else:
            ch= 512
        if resl==2:
            self.conv1 = nn.Sequential()
        else:
            self.conv1 = conv1(resl)
        self.noise1 = Noise()
        self.adain1 = AdaptiveInstanceNorm(ch) #h와 ch를 착각함.
        self.conv2 = conv2(resl)
        self.noise2 = Noise()
        self.adain2 = AdaptiveInstanceNorm(ch)
        
    def forward(self,x,style_latent): 
        print('=====styleConvBlock==')
        y = self.conv1(x)
        y = self.noise1(y)
        y = self.adain1(y,style_latent) 
        y = self.conv2(y)
        y = self.noise2(y)
        y = self.adain2(y,style_latent)
        
        return y

In [78]:
dummy = torch.ones([2,512,4,4])
style_latent = torch.ones([1,512])
model = StyleConvBlock(2)

In [79]:
dummy_output = model(dummy,style_latent)

=====styleConvBlock==
noise torch.Size([2, 1, 4, 4])
noise success
x_norm torch.Size([2, 512, 4, 4])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
sample torch.Size([2, 512, 4, 4])
noise torch.Size([2, 1, 4, 4])
noise success
x_norm torch.Size([2, 512, 4, 4])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
sample torch.Size([2, 512, 4, 4])


In [80]:
dummy = torch.ones([2,512,4,4])
style_latent = torch.ones([1,512])
model_resl2 = StyleConvBlock(2)
model_resl3 = StyleConvBlock(3)
model_resl4 = StyleConvBlock(4)
model_resl5 = StyleConvBlock(5)
model_resl6 = StyleConvBlock(6)
model_resl7 = StyleConvBlock(7)
model_resl8 = StyleConvBlock(8)

In [81]:

dummy_output = model_resl2(dummy,style_latent)
dummy_output = model_resl3(dummy_output,style_latent)
dummy_output = model_resl4(dummy_output,style_latent)
dummy_output = model_resl5(dummy_output,style_latent)
dummy_output = model_resl6(dummy_output,style_latent)
dummy_output = model_resl7(dummy_output,style_latent)
dummy_output = model_resl8(dummy_output,style_latent)


=====styleConvBlock==
noise torch.Size([2, 1, 4, 4])
noise success
x_norm torch.Size([2, 512, 4, 4])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
sample torch.Size([2, 512, 4, 4])
noise torch.Size([2, 1, 4, 4])
noise success
x_norm torch.Size([2, 512, 4, 4])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
sample torch.Size([2, 512, 4, 4])
=====styleConvBlock==
noise torch.Size([2, 1, 8, 8])
noise success
x_norm torch.Size([2, 512, 8, 8])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
sample torch.Size([2, 512, 8, 8])
noise torch.Size([2, 1, 8, 8])
noise success
x_norm torch.Size([2, 512, 8, 8])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
sample torch.Size([2, 512, 8, 8])
=====styleConvBlock==
noise torch.Size([2, 1, 16, 16])
noise success
x_norm torch.Size([2, 512, 16, 16])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
samp

In [82]:
print(model_resl8)

StyleConvBlock(
  (conv1): Sequential(
    (0): Upsample(scale_factor=2.0, mode=nearest)
    (1): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (noise1): Noise()
  (adain1): AdaptiveInstanceNorm(
    (instance_norm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (linear): Linear(in_features=512, out_features=128, bias=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (noise2): Noise()
  (adain2): AdaptiveInstanceNorm(
    (instance_norm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (linear): Linear(in_features=512, out_features=128, bias=True)
  )
)


In [93]:
def to_rgb_layer(resl=3):
    if resl>=6:
        ch = 2**(14-resl)
        to_rgb = nn.Conv2d(ch,3,3,1,1)
    else:
        to_rgb = nn.Conv2d(512,3,3,1,1)
    return to_rgb
    

In [84]:
to_rgb = ToRgbBlock(8)
to_rgb(dummy_output,style_latent).shape

=====ToRgbBlock==


torch.Size([2, 3, 256, 256])

In [94]:
style_conv_blocks = [
    StyleConvBlock(resl=2),
    StyleConvBlock(resl=3),
    StyleConvBlock(resl=4),
    StyleConvBlock(resl=5),
    StyleConvBlock(resl=6),
    StyleConvBlock(resl=7),
    StyleConvBlock(resl=8),
]
to_rgb_layers = [
    to_rgb_layer(resl=2),
    to_rgb_layer(resl=3),
    to_rgb_layer(resl=4),
    to_rgb_layer(resl=5),
    to_rgb_layer(resl=6),
    to_rgb_layer(resl=7),
    to_rgb_layer(resl=8),
]
layers =[]

In [99]:
class mySequential(nn.Sequential):
    def forward(self, *inputs):
        for module in self._modules.values():
            if type(inputs) == tuple:
                inputs = module(*inputs)
            else:
                inputs = module(inputs)
        return inputs

In [100]:
iter_num = 1
for i in range(iter_num):
    layers.append(style_conv_blocks[i])
layers.append(to_rgb_layers[iter_num-1])
#mymodel = nn.Sequential(*layers)
mymodel = mySequential()
mymodel.add_module('asdf',style_conv_blocks[0])
mymodel.add_module('qwer',to_rgb_layers[0])


# mymodel2 = nn.Sequential()
# linear1 = nn.Linear(512,128)
# linear2 = nn.Linear(128,8)
# mymodel2.add_module('(0)',linear1)
# mymodel2.add_module('(1)',linear2)
# linear1.add_module('(1)',linear2)


In [101]:
print(mymodel)


mySequential(
  (asdf): StyleConvBlock(
    (conv1): Sequential()
    (noise1): Noise()
    (adain1): AdaptiveInstanceNorm(
      (instance_norm): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (linear): Linear(in_features=512, out_features=1024, bias=True)
    )
    (conv2): Sequential(
      (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (noise2): Noise()
    (adain2): AdaptiveInstanceNorm(
      (instance_norm): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (linear): Linear(in_features=512, out_features=1024, bias=True)
    )
  )
  (qwer): Conv2d(512, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


In [102]:
dummy = torch.ones([2,512,4,4])
style_latent = torch.ones([1,512])
dummy_output = mymodel(dummy,style_latent)
print("============",dummy_output.shape)
# mymodel2(dummy_output,style_latent).shape

StyleConvBlock(
  (conv1): Sequential()
  (noise1): Noise()
  (adain1): AdaptiveInstanceNorm(
    (instance_norm): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (linear): Linear(in_features=512, out_features=1024, bias=True)
  )
  (conv2): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (noise2): Noise()
  (adain2): AdaptiveInstanceNorm(
    (instance_norm): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (linear): Linear(in_features=512, out_features=1024, bias=True)
  )
)
-------------------
=====styleConvBlock==
noise torch.Size([2, 1, 4, 4])
noise success
x_norm torch.Size([2, 512, 4, 4])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN success
sample torch.Size([2, 512, 4, 4])
noise torch.Size([2, 1, 4, 4])
noise success
x_norm torch.Size([2, 512, 4, 4])
gamma torch.Size([1, 512, 1, 1])
beta torch.Size([1, 512, 1, 1])
AdaIN succ

## block 연결해서 완성하기

In [116]:
# Style Conv Block 을 만들고 연결.
# to_rgb layer 만들고 연결
# to_rgb layer는 상황에 따라 값이 바뀜

class SynthesisGen(nn.Module):
    def __init__(self,resl):
        super().__init__()
        self.style_conv_blocks = [
            StyleConvBlock(resl=2),
            StyleConvBlock(resl=3),
            StyleConvBlock(resl=4),
            StyleConvBlock(resl=5),
            StyleConvBlock(resl=6),
            StyleConvBlock(resl=7),
            StyleConvBlock(resl=8),
        ]
        self.to_rgb_layers = [
            to_rgb_layer(resl=2),
            to_rgb_layer(resl=3),
            to_rgb_layer(resl=4),
            to_rgb_layer(resl=5),
            to_rgb_layer(resl=6),
            to_rgb_layer(resl=7),
            to_rgb_layer(resl=8),
        ]
        self.synthesis_net = self.set_synthesis_network(resl)
    def set_synthesis_network(self,resl=2):
        # st_conv resl-1개수만큼 연결 <- for 문
        # [resl-1] 인덱싱으로 to_rgb 붙이기
        iter_num = resl-1
        layers=[]
        # for i in range(iter_num):
        #     layers.append(self.style_conv_blocks[i])
        layers.append(self.style_conv_blocks[0])
        layers.append(self.to_rgb_layers[iter_num-1])
        return mySequential(*layers)

    def forward(self,x,style_latent_w):
        to_rgb=self.synthesis_net(x,style_latent_w) ##여기가 error
        return to_rgb
        

In [117]:
constant_input = torch.randn((2,512,4,4)) 

In [118]:
class styleFC(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(), # 마지막 activation fn은 뭐지?
        )
    def forward(self,x): 
        #x [1,512]
        return self.fc(x) #[1,512]

In [119]:
class styleGen(nn.Module):
    def __init__(self,resl=3):
        super().__init__()
        self.style_fc = styleFC()
        self.synthesis_gen = SynthesisGen(resl)
        
# style latent W 만들고 연결

    def forward(self,z,constant_input):
        style_latent_w = self.style_fc(z)
        output = self.synthesis_gen(constant_input,style_latent_w)
        #그럼 to rgb 거쳐서 이미지로 나오겠지.
        return output


In [120]:
my_style_gen = styleGen(3)
constant_input_dummy = torch.ones((2,512,4,4))
z = torch.ones((1,512))


In [127]:
du = torch.randn(2,3)

print(du.shape)
du.requires_grad_()

torch.Size([2, 3])


tensor([[-0.1292,  0.4887, -1.8443],
        [ 0.2806, -0.4141, -0.2033]], requires_grad=True)

In [129]:
#dudu =nn.Parameter(torch.zeros(1, 3, 1, 1))
dudu =torch.zeros(1, 3, 1, 1)
print(dudu.shape)
dudu.requires_grad_()

torch.Size([1, 3, 1, 1])


tensor([[[[0.]],

         [[0.]],

         [[0.]]]], requires_grad=True)